In [1]:
# 1
import pandas as pd
water = pd.read_csv("water.txt", sep="\t")
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [2]:
print "Pearson corr = %.4f" % water.mortality.corr(water.hardness)

Pearson corr = -0.6548


In [3]:
# 2
from scipy.stats import spearmanr
print "Spearman corr = %.4f" % spearmanr(water.mortality, water.hardness).correlation

Spearman corr = -0.6317


In [4]:
# 3
north = water[water.location=="North"]
south = water[water.location=="South"]

print "North corr = %.4f" % north.mortality.corr(north.hardness)  # -0.3686
print "South corr = %.4f" % south.mortality.corr(south.hardness)

North corr = -0.3686
South corr = -0.6022


In [6]:
# 4
from math import sqrt

def mcc(a, b, c, d):
    """Mattehews correlation"""
    numerator = a*d-b*c
    denominator = sqrt((a+b)*(a+c)*(b+d)*(c+d))
    return float(numerator) / denominator

a = 239
b = 515
c = 203
d = 718

print "Mattehews correlation = %.3f" % mcc(a, b, c, d)

Mattehews correlation = 0.109


In [7]:
# 5
from scipy.stats import chi2_contingency
g, p, dof, expctd = chi2_contingency([[a, b], [c, d]])
print "p-value =", p  # 5

p-value = 1.05589870066e-05


In [8]:
# 6
import scipy
import numpy as np

def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)
    
men = [1]*a+[0]*b
women = [1]*c+[0]*d

print "95%% confidence interval for a difference between proportions: [%.4f, %.4f]" % \
                                                proportions_diff_confint_ind(men, women)  # 0.0539

95% confidence interval for a difference between proportions: [0.0539, 0.1392]


In [9]:
# 7

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [10]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [11]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(women, men), 
                                              alternative="two-sided")  # 6

p-value: 0.000008


In [12]:
# 8

# first method
def chi_stat(mat):
    n = mat.sum()
    i_sum = mat.sum(axis=1)
    sum_j = mat.sum(axis=0)
    stat = 0
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            stat += (mat[i][j]**2) / float(i_sum[i]) / sum_j[j]
    return n * (stat - 1)

In [13]:
# second method
def chi_stat2(mat):
    n = mat.sum()
    i_sum = mat.sum(axis=1)
    sum_j = mat.sum(axis=0)
    stat = 0
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            stat += (mat[i][j] - i_sum[i]*sum_j[j]/float(n))**2 / (i_sum[i]*sum_j[j]/float(n))
    return stat

In [14]:
mat = np.array([[197, 111, 33],
       [382, 685, 331],
       [110, 342, 333]])
print "Chi-stat = %.4f" % chi_stat(mat)

Chi-stat = 293.6831


In [15]:
print "Chi-stat = %.4f" % chi_stat2(mat)

Chi-stat = 293.6831


In [16]:
# 9
# third method
g, p, dof, expctd = chi2_contingency(mat)
print "Chi-stat = %.4f" % g
print "p-value =", p  # 62

Chi-stat = 293.6831
p-value = 2.49642995801e-62


In [112]:
# 10
print "Cramers V = %.4f" % sqrt(float(g)/mat.sum()/(min(mat.shape[0], mat.shape[1])-1))

Cramers V = 0.2412
